# Working with Geometries

## Introduction

This notebook demonstrates how to work with geometries in DuckDB.

## Installation

Uncomment the following cell to install the required packages if needed.

In [1]:
%pip install duckdb leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.5/765.5 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.9 MB/s eta 0:00:00


## Library Import and Configuration

In [3]:
import duckdb
import leafmap

## Sample Data

The datasets in the database are in NAD83 / UTM zone 18N projection, EPSG:26918.

In [4]:
url = "https://open.gishub.org/data/duckdb/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://open.gishub.org/data/duckdb/nyc_data.db.zip
To: /content/nyc_data.db.zip
100%|██████████| 8.60M/8.60M [00:00<00:00, 78.8MB/s]


Extracting files...


'/content/nyc_data.db.zip'

## Connecting to DuckDB

Connect jupysql to DuckDB using a SQLAlchemy-style connection string. You may either connect to an in memory DuckDB, or a file backed db.

In [5]:
con = duckdb.connect("nyc_data.db")

In [6]:
con.install_extension("spatial")
con.load_extension("spatial")

In [7]:
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

## Creating samples

In [8]:
con.sql("""

CREATE or REPLACE TABLE samples (name VARCHAR, geom GEOMETRY);

INSERT INTO samples VALUES
  ('Point', ST_GeomFromText('POINT(-100 40)')),
  ('Linestring', ST_GeomFromText('LINESTRING(0 0, 1 1, 2 1, 2 2)')),
  ('Polygon', ST_GeomFromText('POLYGON((0 0, 1 0, 1 1, 0 1, 0 0))')),
  ('PolygonWithHole', ST_GeomFromText('POLYGON((0 0, 10 0, 10 10, 0 10, 0 0),(1 1, 1 2, 2 2, 2 1, 1 1))')),
  ('Collection', ST_GeomFromText('GEOMETRYCOLLECTION(POINT(2 0),POLYGON((0 0, 1 0, 1 1, 0 1, 0 0)))'));

SELECT * FROM samples;

  """)

┌─────────────────┬───────────────────────────────────────────────────────────────────────┐
│      name       │                                 geom                                  │
│     varchar     │                               geometry                                │
├─────────────────┼───────────────────────────────────────────────────────────────────────┤
│ Point           │ POINT (-100 40)                                                       │
│ Linestring      │ LINESTRING (0 0, 1 1, 2 1, 2 2)                                       │
│ Polygon         │ POLYGON ((0 0, 1 0, 1 1, 0 1, 0 0))                                   │
│ PolygonWithHole │ POLYGON ((0 0, 10 0, 10 10, 0 10, 0 0), (1 1, 1 2, 2 2, 2 1, 1 1))    │
│ Collection      │ GEOMETRYCOLLECTION (POINT (2 0), POLYGON ((0 0, 1 0, 1 1, 0 1, 0 0))) │
└─────────────────┴───────────────────────────────────────────────────────────────────────┘

In [9]:
con.sql("SELECT name, ST_AsText(geom) AS geometry FROM samples;")

┌─────────────────┬───────────────────────────────────────────────────────────────────────┐
│      name       │                               geometry                                │
│     varchar     │                                varchar                                │
├─────────────────┼───────────────────────────────────────────────────────────────────────┤
│ Point           │ POINT (-100 40)                                                       │
│ Linestring      │ LINESTRING (0 0, 1 1, 2 1, 2 2)                                       │
│ Polygon         │ POLYGON ((0 0, 1 0, 1 1, 0 1, 0 0))                                   │
│ PolygonWithHole │ POLYGON ((0 0, 10 0, 10 10, 0 10, 0 0), (1 1, 1 2, 2 2, 2 1, 1 1))    │
│ Collection      │ GEOMETRYCOLLECTION (POINT (2 0), POLYGON ((0 0, 1 0, 1 1, 0 1, 0 0))) │
└─────────────────┴───────────────────────────────────────────────────────────────────────┘

In [10]:
con.sql("""

COPY samples TO 'samples.geojson' (FORMAT GDAL, DRIVER GeoJSON);

""")

## Points

![](https://postgis.net/workshops/postgis-intro/_images/points.png)

A spatial point represents a single location on the Earth. This point is represented by a single coordinate (including either 2-, 3- or 4-dimensions). Points are used to represent objects when the exact details, such as shape and size, are not important at the target scale. For example, cities on a map of the world can be described as points, while a map of a single state might represent cities as polygons.


In [11]:
con.sql("""

SELECT ST_AsText(geom)
  FROM samples
  WHERE name = 'Point';

""")

┌─────────────────┐
│ st_astext(geom) │
│     varchar     │
├─────────────────┤
│ POINT (-100 40) │
└─────────────────┘

Some of the specific spatial functions for working with points are:

- **ST_X(geom)** returns the X ordinate
- **ST_Y(geom)** returns the Y ordinate

So, we can read the ordinates from a point like this:

In [12]:
con.sql("""

SELECT ST_X(geom), ST_Y(geom)
  FROM samples
  WHERE name = 'Point';

""")

┌────────────┬────────────┐
│ st_x(geom) │ st_y(geom) │
│   double   │   double   │
├────────────┼────────────┤
│     -100.0 │       40.0 │
└────────────┴────────────┘

In [13]:
con.sql("""

SELECT * FROM nyc_subway_stations

""")

┌──────────┬────────┬───────────────────┬─────────────────┬─────────────────┬─────────────────────────────────────────┬───────────────────────────────────┬───────────┬─────────┬─────────┬───────────┬──────────────┬─────────┬─────────┬──────────────────────────────────────────────┐
│ OBJECTID │   ID   │       NAME        │    ALT_NAME     │    CROSS_ST     │                LONG_NAME                │               LABEL               │  BOROUGH  │ NGHBHD  │ ROUTES  │ TRANSFERS │    COLOR     │ EXPRESS │ CLOSED  │                     geom                     │
│  double  │ double │      varchar      │     varchar     │     varchar     │                 varchar                 │              varchar              │  varchar  │ varchar │ varchar │  varchar  │   varchar    │ varchar │ varchar │                   geometry                   │
├──────────┼────────┼───────────────────┼─────────────────┼─────────────────┼─────────────────────────────────────────┼───────────────────────────────────

In [14]:
con.sql("""

SELECT name, ST_AsText(geom)
  FROM nyc_subway_stations
  LIMIT 10;

""")

┌──────────────┬─────────────────────────────────────────────┐
│     NAME     │               st_astext(geom)               │
│   varchar    │                   varchar                   │
├──────────────┼─────────────────────────────────────────────┤
│ Cortlandt St │ POINT (583521.854408956 4507077.862599085)  │
│ Rector St    │ POINT (583324.4866324601 4506805.373160211) │
│ South Ferry  │ POINT (583304.1823994748 4506069.654048115) │
│ 138th St     │ POINT (590250.10594797 4518558.019924332)   │
│ 149th St     │ POINT (590454.7399891173 4519145.719617855) │
│ 149th St     │ POINT (590465.8934191109 4519168.697483203) │
│ 161st St     │ POINT (590573.169495527 4520214.766177284)  │
│ 167th St     │ POINT (591252.8314104103 4520950.353355553) │
│ 167th St     │ POINT (590946.3972262995 4521077.318976877) │
│ 170th St     │ POINT (591583.6111452815 4521434.846626811) │
├──────────────┴─────────────────────────────────────────────┤
│ 10 rows                                          2 co

## Linestrings

![](https://postgis.net/workshops/postgis-intro/_images/lines.png)


A **linestring** is a path between locations. It takes the form of an
ordered series of two or more points. Roads and rivers are typically
represented as linestrings. A linestring is said to be **closed** if it
starts and ends on the same point. It is said to be **simple** if it
does not cross or touch itself (except at its endpoints if it is
closed). A linestring can be both **closed** and **simple**.

The street network for New York (`nyc_streets`) was loaded earlier in
the workshop. This dataset contains details such as name, and type. A
single real world street may consist of many linestrings, each
representing a segment of road with different attributes.

The following SQL query will return the geometry associated with one
linestring (in the `ST_AsText` column).

In [15]:
con.sql("""

SELECT ST_AsText(geom)
  FROM samples
  WHERE name = 'Linestring';

""")

┌─────────────────────────────────┐
│         st_astext(geom)         │
│             varchar             │
├─────────────────────────────────┤
│ LINESTRING (0 0, 1 1, 2 1, 2 2) │
└─────────────────────────────────┘

Some of the specific spatial functions for working with linestrings are:

-   `ST_Length(geom)` returns the length of the linestring
-   `ST_StartPoint(geom)` returns the first coordinate as a point
-   `ST_EndPoint(geom)` returns the last coordinate as a point
-   `ST_NPoints(geom)` returns the number of coordinates in the
    linestring

So, the length of our linestring is:

In [16]:
con.sql("""

SELECT ST_Length(geom)
  FROM samples
  WHERE name = 'Linestring';

""")

┌───────────────────┐
│  st_length(geom)  │
│      double       │
├───────────────────┤
│ 3.414213562373095 │
└───────────────────┘

## Polygons

![](https://postgis.net/workshops/postgis-intro/_images/polygons.png)

A polygon is a representation of an area. The outer boundary of the
polygon is represented by a ring. This ring is a linestring that is both
closed and simple as defined above. Holes within the polygon are also
represented by rings.

Polygons are used to represent objects whose size and shape are
important. City limits, parks, building footprints or bodies of water
are all commonly represented as polygons when the scale is sufficiently
high to see their area. Roads and rivers can sometimes be represented as
polygons.

The following SQL query will return the geometry associated with one
polygon (in the `ST_AsText` column).

In [17]:
con.sql("""

SELECT ST_AsText(geom)
  FROM samples
  WHERE name LIKE 'Polygon%';

""")

┌────────────────────────────────────────────────────────────────────┐
│                          st_astext(geom)                           │
│                              varchar                               │
├────────────────────────────────────────────────────────────────────┤
│ POLYGON ((0 0, 1 0, 1 1, 0 1, 0 0))                                │
│ POLYGON ((0 0, 10 0, 10 10, 0 10, 0 0), (1 1, 1 2, 2 2, 2 1, 1 1)) │
└────────────────────────────────────────────────────────────────────┘

Some of the specific spatial functions for working with polygons are:

-   `ST_Area(geom)` returns the area of the polygons
-   `ST_NRings(geom)` returns the number of rings (usually 1, more
    of there are holes)
-   `ST_ExteriorRing(geom)` returns the outer ring as a linestring
-   `ST_InteriorRingN(geometry,n)` returns a specified interior ring as
    a linestring
-   `ST_Perimeter(geom)` returns the length of all the rings

We can calculate the area of our polygons using the area function:

In [18]:
con.sql("""

SELECT name, ST_Area(geom)
  FROM samples
  WHERE name LIKE 'Polygon%';

""")

┌─────────────────┬───────────────┐
│      name       │ st_area(geom) │
│     varchar     │    double     │
├─────────────────┼───────────────┤
│ Polygon         │           1.0 │
│ PolygonWithHole │          99.0 │
└─────────────────┴───────────────┘

## Collections

There are four collection types, which group multiple simple samples
into sets.

-   **MultiPoint**, a collection of points
-   **MultiLineString**, a collection of linestrings
-   **MultiPolygon**, a collection of polygons
-   **GeometryCollection**, a heterogeneous collection of any geometry
    (including other collections)

Collections are another concept that shows up in GIS software more than
in generic graphics software. They are useful for directly modeling real
world objects as spatial objects. For example, how to model a lot that
is split by a right-of-way? As a **MultiPolygon**, with a part on either
side of the right-of-way.

Our example collection contains a polygon and a point:

In [19]:
con.sql("""

SELECT name, ST_AsText(geom)
  FROM samples
  WHERE name = 'Collection';

""")

┌────────────┬───────────────────────────────────────────────────────────────────────┐
│    name    │                            st_astext(geom)                            │
│  varchar   │                                varchar                                │
├────────────┼───────────────────────────────────────────────────────────────────────┤
│ Collection │ GEOMETRYCOLLECTION (POINT (2 0), POLYGON ((0 0, 1 0, 1 1, 0 1, 0 0))) │
└────────────┴───────────────────────────────────────────────────────────────────────┘

## Data Visualization

In [20]:
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
│ samples             │
└─────────────────────┘

In [21]:
subway_stations_df = con.sql("SELECT * EXCLUDE geom, ST_AsText(geom) as geometry FROM nyc_subway_stations").df()
subway_stations_df.head()

,OBJECTID,ID,NAME,ALT_NAME,CROSS_ST,LONG_NAME,LABEL,BOROUGH,NGHBHD,ROUTES,TRANSFERS,COLOR,EXPRESS,CLOSED,geometry
0,1.0,376.0,Cortlandt St,None,Church St,"Cortlandt St (R,W) Manhattan","Cortlandt St (R,W)",Manhattan,None,"R,W","R,W",YELLOW,None,None,POINT (583521.854408956 4507077.862599085)
1,2.0,2.0,Rector St,None,None,Rector St (1) Manhattan,Rector St (1),Manhattan,None,1,1,RED,None,None,POINT (583324.4866324601 4506805.373160211)
2,3.0,1.0,South Ferry,None,None,South Ferry (1) Manhattan,South Ferry (1),Manhattan,None,1,1,RED,None,None,POINT (583304.1823994748 4506069.654048115)
3,4.0,125.0,138th St,Grand Concourse,Grand Concourse,"138th St / Grand Concourse (4,5) Bronx","138th St / Grand Concourse (4,5)",Bronx,None,"4,5","4,5",GREEN,None,None,POINT (590250.10594797 4518558.019924332)
4,5.0,126.0,149th St,Grand Concourse,Grand Concourse,149th St / Grand Concourse (4) Bronx,149th St / Grand Concourse (4),Bronx,None,4,"2,4,5",GREEN,express,None,POINT (590454.7399891173 4519145.719617855)


In [22]:
subway_stations_gdf = leafmap.df_to_gdf(subway_stations_df, src_crs="EPSG:26918", dst_crs="EPSG:4326")
subway_stations_gdf.head()

,OBJECTID,ID,NAME,ALT_NAME,CROSS_ST,LONG_NAME,LABEL,BOROUGH,NGHBHD,ROUTES,TRANSFERS,COLOR,EXPRESS,CLOSED,geometry
0,1.0,376.0,Cortlandt St,None,Church St,"Cortlandt St (R,W) Manhattan","Cortlandt St (R,W)",Manhattan,None,"R,W","R,W",YELLOW,None,None,POINT (-74.01122 40.71038)
1,2.0,2.0,Rector St,None,None,Rector St (1) Manhattan,Rector St (1),Manhattan,None,1,1,RED,None,None,POINT (-74.01359 40.70795)
2,3.0,1.0,South Ferry,None,None,South Ferry (1) Manhattan,South Ferry (1),Manhattan,None,1,1,RED,None,None,POINT (-74.01393 40.70132)
3,4.0,125.0,138th St,Grand Concourse,Grand Concourse,"138th St / Grand Concourse (4,5) Bronx","138th St / Grand Concourse (4,5)",Bronx,None,"4,5","4,5",GREEN,None,None,POINT (-73.92992 40.81308)
4,5.0,126.0,149th St,Grand Concourse,Grand Concourse,149th St / Grand Concourse (4) Bronx,149th St / Grand Concourse (4),Bronx,None,4,"2,4,5",GREEN,express,None,POINT (-73.92741 40.81835)


In [23]:
subway_stations_gdf.explore()

ImportError: The 'folium', 'matplotlib' and 'mapclassify' packages are required for 'explore()'. You can install them using 'conda install -c conda-forge folium matplotlib mapclassify' or 'pip install folium matplotlib mapclassify'.

In [24]:
nyc_streets_df = con.sql("SELECT * EXCLUDE geom, ST_AsText(geom) as geometry FROM nyc_streets").df()
nyc_streets_df.head()

,ID,NAME,ONEWAY,TYPE,geometry
0,1,Shore Pky S,None,residential,MULTILINESTRING ((586785.4767897038 4492901.00...
1,2,None,None,footway,MULTILINESTRING ((586645.0073625665 4504977.75...
2,3,Avenue O,None,residential,MULTILINESTRING ((586750.3019977848 4496109.72...
3,4,Walsh Ct,None,residential,MULTILINESTRING ((586728.695515043 4497971.053...
4,5,None,None,motorway_link,MULTILINESTRING ((586587.0531467082 4510088.25...


In [25]:
nyc_streets_gdf = leafmap.df_to_gdf(nyc_streets_df, src_crs="EPSG:26918", dst_crs="EPSG:4326")
nyc_streets_gdf.head()

,ID,NAME,ONEWAY,TYPE,geometry
0,1,Shore Pky S,None,residential,"MULTILINESTRING ((-73.97454 40.58235, -73.9732..."
1,2,None,None,footway,"MULTILINESTRING ((-73.97454 40.69115, -73.9743..."
2,3,Avenue O,None,residential,"MULTILINESTRING ((-73.97451 40.61126, -73.9734..."
3,4,Walsh Ct,None,residential,"MULTILINESTRING ((-73.97451 40.62803, -73.9726..."
4,5,None,None,motorway_link,"MULTILINESTRING ((-73.97452 40.73718, -73.9738..."


In [26]:
nyc_streets_gdf.explore()

ImportError: The 'folium', 'matplotlib' and 'mapclassify' packages are required for 'explore()'. You can install them using 'conda install -c conda-forge folium matplotlib mapclassify' or 'pip install folium matplotlib mapclassify'.